<a href="https://colab.research.google.com/github/NauvalZulfikar/byScript/blob/main/POST_Crypto_1H.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LIBRARY

In [ ]:
import requests
import pandas as pd
import datetime
import numpy as np
import time
import math
import random
import seaborn as sns

import gspread
from google.auth import default

def post_data_in_chunks(df, coin, post_url, headers, chunk_size=5000):
    """Splits large dataframes into smaller chunks and posts them to the API."""

    num_chunks = math.ceil(len(df) / chunk_size)

    for i in range(num_chunks):
        chunk = df.iloc[i * chunk_size: (i + 1) * chunk_size].copy()
        chunk["Datetime"] = chunk["Datetime"].astype(str)  # Convert Timestamp to string

        json_data = {f"{coin}USDT": chunk.to_dict(orient="records")}

        try:
            post_response = requests.post(post_url, json=json_data, headers=headers)

            if post_response.status_code in [200, 201]:
                print(f"✅ Successfully posted chunk {i+1}/{num_chunks} for {coin}")
            else:
                print(f"⚠️ Error posting chunk {i+1}/{num_chunks} for {coin}: {post_response.status_code}, {post_response.text}")

        except Exception as e:
            print(f"🚨 Exception posting chunk {i+1}/{num_chunks} for {coin}: {e}")

        time.sleep(1)  # Prevent excessive API requests

def fetch_and_post_data(coin, API_KEY):
    BASE_URL = "https://min-api.cryptocompare.com/data/v2/histohour"
    start_date = "2017-01-01 00:00:00"
    end_date = "2025-12-31 23:59:59"

    adjusted_start_date = pd.Timestamp(start_date) - pd.Timedelta(hours=2007)  # 2000 + 7 hours
    start_timestamp = int(pd.Timestamp(adjusted_start_date).timestamp())
    end_timestamp = int(pd.Timestamp(end_date).timestamp())

    all_data = []
    current_timestamp = end_timestamp

    try:
        while current_timestamp > start_timestamp:
            params = {
                "fsym": coin,
                "tsym": "USDT",
                "limit": 2000,
                "aggregate": 1,
                "toTs": current_timestamp,
                "e": "Binance"
            }

            response = requests.get(BASE_URL, headers={"authorization": f"Apikey {API_KEY}"}, params=params)

            if response.status_code == 200:
                data = response.json().get("Data", {}).get("Data", [])
                if not data:
                    break

                all_data.extend(data)
                current_timestamp = data[0]["time"]
                time.sleep(1)  # Prevent hitting API rate limits

            else:
                print(f"⚠️ Error fetching data for {coin}: {response.status_code}, {response.text}")
                return None

    except Exception as e:
        print(f"🚨 Exception fetching data for {coin}: {e}")
        return None

    if not all_data:
        print(f"⚠️ No data retrieved for {coin}. Exiting.")
        return None

    # Convert collected data into a DataFrame
    df = pd.DataFrame(all_data)

    # Convert UNIX timestamp to datetime (UTC first, then UTC+7)
    df["Datetime"] = pd.to_datetime(df["time"], unit="s", utc=True).dt.tz_convert("Asia/Bangkok")
    df.drop(columns=["time"], inplace=True)

    # Rename columns for clarity
    df.rename(columns={"volumefrom": f"volume_{coin}", "volumeto": "volume_USDT"}, inplace=True)

    # Sort by Datetime
    df = df.sort_values("Datetime").reset_index(drop=True)

    # Remove consecutive duplicate rows (excluding "Datetime")
    df_cleaned = df.loc[df.drop(columns=["Datetime"]).apply(tuple, axis=1).shift() != df.drop(columns=["Datetime"]).apply(tuple, axis=1)]
    df_cleaned = df_cleaned.reset_index(drop=True).iloc[1:].reset_index(drop=True)

    if df_cleaned.empty:
        print(f"⚠️ No valid data to send for {coin}. Skipping API request.")
        return None

    # API POST request to byscript.io
    post_url = "https://byscript.io/api/feed-data/"
    headers = {"accept": "application/json", "Content-type": "application/json"}

    post_data_in_chunks(df_cleaned, coin, post_url, headers, chunk_size=5000)

    print(f"✅ Finished processing {coin}")

    return df_cleaned

top_100_crypto_symbols = [
    'BTC', 'ETH', 'XRP', 'USDT', 'BNB', 'SOL', 'USDC', 'DOGE', 'ADA', 'TRX',
    'LINK', 'SUI', 'AVAX', 'XLM', 'LTC', 'HBAR', 'SHIB', 'LEO', 'TON', 'HYPE',
    'DOT', 'OM', 'BCH', 'USDe', 'UNI', 'BGB', 'DAI', 'XMR', 'NEAR', 'PEPE',
    'AAVE', 'TAO', 'ONDO', 'APT', 'MNT', 'ICP', 'TRUMP', 'ETC', 'OKB', 'KAS',
    'S', 'VET', 'POL', 'RNDR', 'ALGO', 'CRO', 'FIL', 'ARB', 'JUP', 'GT',
    'FDUSD', 'TIA', 'ATOM', 'OP', 'FET', 'LDO', 'DEXE', 'INJ', 'KCS', 'STX',
    'XDC', 'RAY', 'THETA', 'GRT', 'SEI', 'IMX', 'ENA', 'WLD', 'BONK', 'IP',
    'MOVE', 'MKR', 'QNT', 'FLR', 'JASMY', 'EOS', 'ENS', 'JTO', 'FLOKI', 'XTZ',
    'SAND', 'PYTH', 'BERA', 'NEXO', 'GALA', 'BTT', 'IOTA', 'FLOW', 'KAIA',
    'RON', 'NEO', 'BSV', 'CAKE', 'VIRTUAL', 'XAUt', 'AXS', 'PYUSD', 'CRV',
    'MELANIA', 'SPX'
]

API_KEY = "05928b70dcbeba8844727ad0142f8b86e92fd5d6a3a29881d1cd3300d2c27ecf"
for i in top_100_crypto_symbols:
  coin = i
  fetch_and_post_data(coin, API_KEY)